In [ ]:
!git clone https://github.com/syrgkanislab/npiv_functionals.git

Cloning into 'npiv_functionals'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 203 (delta 103), reused 144 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (203/203), 706.58 KiB | 6.42 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [ ]:
%cd npiv_functionals

/content/npiv_functionals


In [ ]:
!python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [ ]:
import os
import sys
sys.path.append(os.path.abspath('.'))

In [ ]:
import warnings
warnings.simplefilter('ignore')
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mliv.dgps import get_data, get_tau_fn, fn_dict
from mliv.neuralnet.utilities import mean_ci
from mliv.neuralnet import AGMMEarlyStop as AGMM
from mliv.neuralnet.moments import avg_small_diff
from sklearn.ensemble import RandomForestRegressor
import joblib
from joblib import Parallel, delayed
from mliv.cct.mc2 import MC2
from mliv.rkhs import ApproxRKHSIVCV
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import FeatureUnion, Pipeline
import scipy

In [ ]:
# average finite difference moment
def moment_fn(x, test_fn):
    epsilon = 0.1
    t1 = np.hstack([x[:, [0]] + epsilon, x[:, 1:]])
    t0 = np.hstack([x[:, [0]] - epsilon, x[:, 1:]])
    return (test_fn(t1) - test_fn(t0)) / (2 * epsilon)

In [ ]:
def moment_evals(x):
    epsilon = 0.1
    t1 = np.hstack([x[:, [0]] + epsilon, x[:, 1:]])
    t0 = np.hstack([x[:, [0]] - epsilon, x[:, 1:]])
    return t0, t1

In [ ]:
it = 0
n = 5000
mc2_gen = MC2(n, 100, None, dimension=10, corr=0.5)
npvec, *_ = mc2_gen.data(it)
Z, X, Y = npvec['instrument'], npvec['endogenous'], npvec['response']
n_z = Z.shape[1]
n_x = X.shape[1]

In [ ]:
Z_train, Z_val, X_train, X_val, Y_train, Y_val = train_test_split(
        Z, X, Y, test_size=.5, shuffle=True)

ztrans = Nystroem(n_components=100)
xtrans = Nystroem(n_components=100)
# ztrans = PolynomialFeatures(degree=2)
# xtrans = PolynomialFeatures(degree=2)
# ztrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=10))])
# xtrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=10))])
ztrans = Pipeline([('trans', ztrans), ('scale', StandardScaler())])
xtrans = Pipeline([('trans', xtrans), ('scale', StandardScaler())])
Psi = ztrans.fit_transform(Z_train)
xtrans.fit(np.vstack((X_train,) + moment_evals(X_train)))
Phi = xtrans.transform(X_train)
mPhi = moment_fn(X_train, xtrans.transform)

In [ ]:
CovPsi = Psi.T @ Psi
CovPhiPsi = Phi.T @ Psi
Phival = xtrans.transform(X_val)
Psival = ztrans.transform(Z_val)
mPhival = moment_fn(X_val, xtrans.transform)
moment_val = np.mean(mPhival, axis=0)

best_violation = np.inf
for alpha in np.logspace(-6, 1, 5):
    regCov = scipy.linalg.pinv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
    Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
    for beta in np.logspace(-6, 1, 5):
        xi = scipy.linalg.pinv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
        for gamma in np.logspace(-6, 1, 5):
            qparam = scipy.linalg.pinv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

            representer_val = np.mean((Psival @ qparam).reshape(-1, 1) * Phival, axis=0)
            violation = np.linalg.norm(moment_val - representer_val, ord=2)
            if violation <= best_violation:
                best_alpha = alpha
                best_beta = beta
                best_gamma = gamma
                best_violation = violation

In [ ]:
alpha = best_alpha
beta = best_beta
gamma = best_gamma
regCov = scipy.linalg.pinv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
xi = scipy.linalg.pinv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
qparam = scipy.linalg.pinv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

In [ ]:
best_alpha, best_beta, best_gamma, best_violation

(1e-06, 0.0031622776601683794, 0.0031622776601683794, 1.0404707570575489)

In [ ]:
agmm = ApproxRKHSIVCV(n_components=200)
agmm.fit(Z_train, X_train, Y_train)

In [ ]:
direct = moment_fn(X_val, agmm.predict).flatten()
residual = (Y_val - agmm.predict(X_val)).flatten()
qvalues = Psival @ qparam
pseudo = direct + qvalues * residual

reg = mean_ci(direct)
dr = mean_ci(pseudo)
ipw = mean_ci(qvalues * Y_val.flatten())
reg, ipw, dr

((1.2141849059398333, 1.0974348660172508, 1.3309349458624158),
 (0.4970061061199458, -0.3013469088314763, 1.295359121071368),
 (1.1251425038796286, 0.850870651162335, 1.3994143565969221))

In [ ]:
xivalues = xtrans.transform(X_val) @ xi
coef = np.mean(qvalues * residual) / np.mean(qvalues * xivalues)
pseudo_tmle = direct + coef * (mPhival @ xi)
pseudo_tmle += qvalues * (residual - coef * xivalues)
tmle = mean_ci(pseudo_tmle)
tmle

(1.116385622031688, 0.8455549641215679, 1.387216279941808)

In [ ]:
def exp_sample_split(it, n, dim, corr):
    mc2_gen = MC2(n, 100, None, dimension=dim, corr=corr)
    npvec, *_ = mc2_gen.data(it)
    Z, X, Y = npvec['instrument'], npvec['endogenous'], npvec['response']
    n_z = Z.shape[1]
    n_x = X.shape[1]

    Z_train, Z_val, X_train, X_val, Y_train, Y_val = train_test_split(
        Z, X, Y, test_size=.5, shuffle=True)

    ztrans = PolynomialFeatures(degree=3)
    xtrans = PolynomialFeatures(degree=3)
    #     ztrans = Nystroem(n_components=100)
    #     xtrans = Nystroem(n_components=100)
    #     ztrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=5))])
    #     xtrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=5))])
    ztrans = Pipeline([('trans', ztrans), ('scale', StandardScaler())])
    xtrans = Pipeline([('trans', xtrans), ('scale', StandardScaler())])

    Psi = ztrans.fit_transform(Z_train)
    Phi = xtrans.fit_transform(X_train)
    mPhi = moment_fn(X_train, xtrans.transform)

    CovPsi = Psi.T @ Psi
    CovPhiPsi = Phi.T @ Psi
    Phival = xtrans.transform(X_val)
    Psival = ztrans.transform(Z_val)
    mPhival = moment_fn(X_val, xtrans.transform)
    moment_val = np.mean(mPhival, axis=0)

    best_violation = np.inf
    for alpha in np.logspace(-6, 1, 5):
        regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
        Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
        for beta in np.logspace(-6, 1, 5):
            xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
            for gamma in np.logspace(-6, 1, 5):
                qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

                # calculating the violation in the riesz representation property for each feature
                #  E[m(W; phi)] = E[q(Z) * phi(X)]
                # for every feature phi.
                representer_val = np.mean((Psival @ qparam).reshape(-1, 1) * Phival, axis=0)
                violation = np.linalg.norm(moment_val - representer_val, ord=2)
                if violation <= best_violation:
                    best_alpha = alpha
                    best_beta = beta
                    best_gamma = gamma
                    best_violation = violation

    alpha = best_alpha
    beta = best_beta
    gamma = best_gamma
    regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
    Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
    xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
    qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

    agmm = ApproxRKHSIVCV(n_components=200)
    agmm.fit(Z_train, X_train, Y_train)

    direct = moment_fn(X_val, agmm.predict).flatten()
    residual = (Y_val - agmm.predict(X_val)).flatten()
    qvalues = Psival @ qparam
    pseudo = direct + qvalues * residual

    reg = mean_ci(direct)
    dr = mean_ci(pseudo)
    ipw = mean_ci(qvalues * Y_val.flatten())

    xivalues = Phival @ xi
    coef = np.mean(qvalues * residual) / np.mean(qvalues * xivalues)
    pseudo_tmle = direct + coef * (mPhival @ xi)
    pseudo_tmle += qvalues * (residual - coef * xivalues)
    tmle = mean_ci(pseudo_tmle)

    return dr, tmle, ipw, reg

In [ ]:
from sklearn.model_selection import KFold

def exp(it, n, dim, corr, fname='cct', iv_strength=None):
    np.random.seed(it)
    if fname == 'cct':
        mc2_gen = MC2(n, 100, None, dimension=dim, corr=corr)
        npvec, *_ = mc2_gen.data(it)
        Z, X, Y = npvec['instrument'], npvec['endogenous'], npvec['response']
    else:
        Z, X, Y, _ = get_data(n, 1, iv_strength, get_tau_fn(fn_dict[fname]), 5)

    direct = np.zeros(n)
    residual = np.zeros(n)
    qvalues = np.zeros(n)
    xivalues = np.zeros(n)
    mxivalues = np.zeros(n)

    for train, test in KFold(n_splits=5, shuffle=True).split(Z):
        Z_train, Z_val, X_train, X_val, Y_train, Y_val = Z[train], Z[test], X[train], X[test], Y[train], Y[test]

        ztrans = Nystroem(n_components=200)
        xtrans = Nystroem(n_components=200)
        ztrans = Pipeline([('trans', ztrans), ('scale', StandardScaler())])
        xtrans = Pipeline([('trans', xtrans), ('scale', StandardScaler())])

        Psi = ztrans.fit_transform(Z_train)
        xtrans.fit(np.vstack((X_train,) + moment_evals(X_train)))
        Phi = xtrans.transform(X_train)
        mPhi = moment_fn(X_train, xtrans.transform)

        CovPsi = Psi.T @ Psi
        CovPhiPsi = Phi.T @ Psi
        Phival = xtrans.transform(X_val)
        Psival = ztrans.transform(Z_val)
        mPhival = moment_fn(X_val, xtrans.transform)
        moment_val = np.mean(mPhival, axis=0)

        best_violation = np.inf
        for alpha in np.logspace(-7, 1, 10):
            regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
            Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
            for beta in np.logspace(-7, 1, 10):
                xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
                for gamma in np.logspace(-7, 1, 10):
                    qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

                    # calculating the violation in the riesz representation property for each feature
                    #  E[m(W; phi)] = E[q(Z) * phi(X)]
                    # for every feature phi.
                    representer_val = np.mean((Psival @ qparam).reshape(-1, 1) * Phival, axis=0)
                    violation = np.linalg.norm(moment_val - representer_val, ord=2)
                    if violation <= best_violation:
                        best_alpha = alpha
                        best_beta = beta
                        best_gamma = gamma
                        best_violation = violation

        alpha = best_alpha
        beta = best_beta
        gamma = best_gamma
        regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
        Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
        xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
        qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

        agmm = ApproxRKHSIVCV(n_components=200)
        agmm.fit(Z_train, X_train, Y_train)

        direct[test] = moment_fn(X_val, agmm.predict).flatten()
        residual[test] = (Y_val - agmm.predict(X_val)).flatten()
        qvalues[test] = Psival @ qparam
        xivalues[test] = Phival @ xi
        mxivalues[test] = mPhival @ xi

    pseudo = direct + qvalues * residual

    reg = mean_ci(direct)
    dr = mean_ci(pseudo)
    ipw = mean_ci(qvalues * Y.flatten())

    coef = np.mean(qvalues * residual) / np.mean(qvalues * xivalues)
    pseudo_tmle = direct + coef * mxivalues
    pseudo_tmle += qvalues * (residual - coef * xivalues)
    tmle = mean_ci(pseudo_tmle)

    return dr, tmle, ipw, reg

In [ ]:
def get_result_dict(results, true, alpha=0.99):
    df = {}
    for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
        if method == 'ipw':
            continue
        data = np.array([r[it] for r in results])
        confidence = .95
        se = (data[:, 2] - data[:, 0]) / scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        confidence = alpha
        data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        if method in ['dr', 'tmle']:
            cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
        else:
            cov = 'NA'
        df[method] =  {'cov': cov,
                'rmse': f'{np.sqrt(np.mean((data[:, 0] - true)**2)):.3f}',
                'bias': f'{np.abs(np.mean((data[:, 0] - true))):.3f}',
                'std': f'{np.std(data[:, 0]):.3f}'}
    return df

In [ ]:
import pandas as pd

true = 1.0

for n in [1000, 5000]:
    for n_x in [0, 5, 10]:
        for corr in [0.0, 0.5]:
            if n_x == 0 and corr == 0.5:
                continue
            print(n, n_x, corr)
            results = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, n_x, corr)
                                                            for it in range(100))
            joblib.dump(results, f'rkhs_cct_n_{n}_n_x_{n_x}_corr_{corr}.jbl')
            df = pd.DataFrame(get_result_dict(results, true))
            display(df)

1000 0 0.0


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.0min finished


,dr,tmle,direct
cov,97,97,NA
rmse,0.083,0.198,0.051
bias,0.004,0.046,0.018
std,0.082,0.193,0.047


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


1000 5 0.0


...

,dr,tmle,direct
cov,96,90,NA
rmse,0.111,0.345,0.106
bias,0.050,0.042,0.082
std,0.100,0.343,0.067


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


5000 10 0.5


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.4min finished


,dr,tmle,direct
cov,95,90,NA
rmse,0.131,0.799,0.144
bias,0.060,0.086,0.114
std,0.116,0.794,0.088


In [31]:
true = 1.0

res = {}
for n_x in [0, 5, 10]:
    res[f'$n_x={n_x}$'] = {}
    for n in [1000, 5000]:
        res[f'$n_x={n_x}$'][f'$n={n}$'] = {}
        for corr in [0.0, 0.5]:
            if n_x == 0 and corr == 0.5:
                continue
            results = joblib.load(f'rkhs_cct_n_{n}_n_x_{n_x}_corr_{corr}.jbl')
            res[f'$n_x={n_x}$'][f'$n={n}$'][f'$\\rho={corr}$'] = pd.DataFrame(get_result_dict(results, true))
        res[f'$n_x={n_x}$'][f'$n={n}$'] = pd.concat(res[f'$n_x={n_x}$'][f'$n={n}$'], sort=False)
    res[f'$n_x={n_x}$'] = pd.concat(res[f'$n_x={n_x}$'], sort=False)
res = pd.concat(res, sort=False).unstack(level=3)

In [32]:
res

dr               tmle                  ipw  \
                             cov   rmse   bias  cov     rmse    bias cov   
$n_x=0$  $n=1000$ $\rho=0.0$  97  0.083  0.004   97    0.198   0.046  NA   
         $n=5000$ $\rho=0.0$  99  0.034  0.011  100    0.105   0.013  NA   
$n_x=5$  $n=1000$ $\rho=0.0$  99  0.141  0.009   97    2.650   0.202  NA   
                  $\rho=0.5$  98  0.158  0.048   92    0.926   0.051  NA   
         $n=5000$ $\rho=0.0$  98  0.086  0.033   91    0.157   0.038  NA   
                  $\rho=0.5$  97  0.088  0.000   93    0.187   0.015  NA   
$n_x=10$ $n=1000$ $\rho=0.0$  99  0.159  0.003   98    1.555   0.232  NA   
                  $\rho=0.5$  94  0.267  0.183   90  435.507  43.277  NA   
         $n=5000$ $\rho=0.0$  96  0.111  0.050   90    0.345   0.042  NA   
                  $\rho=0.5$  95  0.131  0.060   90    0.799   0.086  NA   

                                           direct                
                               rmse   bias    cov   rmse   bias  
$n_x=0$  $n=1000$ $\rho=0.0$  0.167  0.077     NA  0.051  0.018  
         $n=5000$ $\rho=0.0$  0.086  0.057     NA  0.026  0.006  
$n_x=5$  $n=1000$ $\rho=0.0$  0.241  0.130     NA  0.103  0.035  
                  $\rho=0.5$  0.283  0.036     NA  0.104  0.047  
         $n=5000$ $\rho=0.0$  0.122  0.088     NA  0.068  0.037  
                  $\rho=0.5$  0.135  0.051     NA  0.069  0.021  
$n_x=10$ $n=1000$ $\rho=0.0$  0.230  0.131     NA  0.271  0.259  
                  $\rho=0.5$  0.274  0.101     NA  0.106  0.007  
         $n=5000$ $\rho=0.0$  0.124  0.096     NA  0.106  0.082  
                  $\rho=0.5$  0.141  0.071     NA  0.144  0.114

In [33]:
print(res.to_latex(bold_rows=True, multirow=True,
                   multicolumn=True, escape=False,
                   column_format='lll||llll|llll|llll|',
                   multicolumn_format='c|'))

\begin{tabular}{lll||llll|llll|llll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
\multirow[t]{2}{*}{\textbf{$n_x=0$}} & \textbf{$n=1000$} & \textbf{$\rho=0.0$} & 97 & 0.083 & 0.004 & 97 & 0.198 & 0.046 & NA & 0.167 & 0.077 & NA & 0.051 & 0.018 \\
\cline{2-15}
\textbf{} & \textbf{$n=5000$} & \textbf{$\rho=0.0$} & 99 & 0.034 & 0.011 & 100 & 0.105 & 0.013 & NA & 0.086 & 0.057 & NA & 0.026 & 0.006 \\
\cline{1-15} \cline{2-15}
\multirow[t]{4}{*}{\textbf{$n_x=5$}} & \multirow[t]{2}{*}{\textbf{$n=1000$}} & \textbf{$\rho=0.0$} & 99 & 0.141 & 0.009 & 97 & 2.650 & 0.202 & NA & 0.241 & 0.130 & NA & 0.103 & 0.035 \\
\textbf{} & \textbf{} & \textbf{$\rho=0.5$} & 98 & 0.158 & 0.048 & 92 & 0.926 & 0.051 & NA & 0.283 & 0.036 & NA & 0.104 & 0.047 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n=5000$}} & \textb

In [25]:
for fname in ['abs', '2dpoly', 'sigmoid', 'sin']:
    for iv_strength in [.2, .5]:
        Z, X, Y, true_fn = get_data(
            1000000, 1, iv_strength, get_tau_fn(fn_dict[fname]), 5)
        true = np.mean(moment_fn(X, true_fn))
        for n in [500, 1000, 2000]:
            print(n, fname, iv_strength, true)
            results = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, None, None,
                                                                  fname=fname,
                                                                  iv_strength=iv_strength)
                                                            for it in range(100))
            joblib.dump((true, results), f'rkhs_fname_{fname}_n_{n}_iv_strength_{iv_strength}.jbl')
            df = pd.DataFrame(get_result_dict(results, true))
            display(df)

500 abs 0.2 -0.0005001239272161028


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.5min finished


,dr,tmle,direct
cov,85,92,NA
rmse,0.187,0.949,0.159
bias,0.146,0.054,0.137
std,0.117,0.947,0.079


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


1000 abs 0.2 -0.0005001239272161028


...

,dr,tmle,direct
cov,100,99,NA
rmse,0.048,0.715,0.025
bias,0.005,0.080,0.007
std,0.048,0.710,0.025


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


2000 sin 0.5 0.1489351323822737


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.9min finished


,dr,tmle,direct
cov,98,99,NA
rmse,0.041,0.291,0.024
bias,0.011,0.062,0.008
std,0.040,0.285,0.023


In [26]:
def get_result_dict(results, true, alpha=0.99):
    df = {}
    for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
        data = np.array([r[it] for r in results])
        confidence = .95
        se = (data[:, 2] - data[:, 0]) / scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        confidence = alpha
        data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        if method in ['dr', 'tmle']:
            cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
        else:
            cov = 'NA'
        df[method] =  {'cov': cov,
                'rmse': f'{np.sqrt(np.mean((data[:, 0] - true)**2)):.3f}',
                'bias': f'{np.abs(np.mean((data[:, 0] - true))):.3f}'}
    return df

In [27]:
res = {}
for fname in ['abs', '2dpoly', 'sigmoid', 'sin']:
    res[fname] = {}
    for n in [500, 1000, 2000]:
        res[fname][f'$n={n}$'] = {}
        for iv_strength in [.2, .5]:
            true, results = joblib.load(f'rkhs_fname_{fname}_n_{n}_iv_strength_{iv_strength}.jbl')
            df = pd.DataFrame(get_result_dict(results, true))
            res[fname][f'$n={n}$'][f'$\\rho={iv_strength}$'] = df
        res[fname][f'$n={n}$'] = pd.concat(res[fname][f'$n={n}$'], sort=False)
    res[fname] = pd.concat(res[fname], sort=False)
res = pd.concat(res, sort=False).unstack(level=3)

In [28]:
res

dr               tmle               ipw         \
                             cov   rmse   bias  cov   rmse   bias cov   rmse   
abs     $n=500$  $\rho=0.2$   85  0.187  0.146   92  0.949  0.054  NA  0.134   
                 $\rho=0.5$   99  0.088  0.013   98  1.149  0.003  NA  0.078   
        $n=1000$ $\rho=0.2$   91  0.125  0.084   97  1.591  0.241  NA  0.096   
                 $\rho=0.5$  100  0.058  0.003  100  0.826  0.089  NA  0.055   
        $n=2000$ $\rho=0.2$   95  0.084  0.035   96  0.638  0.069  NA  0.071   
                 $\rho=0.5$   97  0.052  0.011  100  0.370  0.078  NA  0.042   
2dpoly  $n=500$  $\rho=0.2$   94  0.174  0.124   97  1.187  0.004  NA  0.230   
                 $\rho=0.5$   96  0.119  0.011   97  1.383  0.025  NA  0.106   
        $n=1000$ $\rho=0.2$   92  0.143  0.081   94  1.140  0.191  NA  0.178   
                 $\rho=0.5$   97  0.070  0.008   99  0.577  0.079  NA  0.078   
        $n=2000$ $\rho=0.2$   95  0.093  0.041   97  0.620  0.082  NA  0.114   
                 $\rho=0.5$   96  0.066  0.018   99  0.370  0.086  NA  0.063   
sigmoid $n=500$  $\rho=0.2$   91  0.152  0.119   94  0.769  0.042  NA  0.086   
                 $\rho=0.5$  100  0.070  0.014   98  0.754  0.018  NA  0.069   
        $n=1000$ $\rho=0.2$   94  0.095  0.059   98  1.108  0.186  NA  0.067   
                 $\rho=0.5$  100  0.048  0.003  100  0.695  0.068  NA  0.048   
        $n=2000$ $\rho=0.2$   96  0.064  0.024   98  0.467  0.036  NA  0.051   
                 $\rho=0.5$   99  0.040  0.008   99  0.295  0.054  NA  0.032   
sin     $n=500$  $\rho=0.2$   87  0.170  0.140   94  0.638  0.075  NA  0.106   
                 $\rho=0.5$  100  0.069  0.003   98  0.882  0.011  NA  0.065   
        $n=1000$ $\rho=0.2$   92  0.106  0.074   96  0.985  0.192  NA  0.076   
                 $\rho=0.5$  100  0.048  0.005   99  0.715  0.080  NA  0.047   
        $n=2000$ $\rho=0.2$   96  0.069  0.032   97  0.491  0.055  NA  0.055   
                 $\rho=0.5$   98  0.041  0.011   99  0.291  0.062  NA  0.032   

                                   direct                
                              bias    cov   rmse   bias  
abs     $n=500$  $\rho=0.2$  0.098     NA  0.159  0.137  
                 $\rho=0.5$  0.012     NA  0.053  0.003  
        $n=1000$ $\rho=0.2$  0.064     NA  0.101  0.085  
                 $\rho=0.5$  0.002     NA  0.036  0.009  
        $n=2000$ $\rho=0.2$  0.028     NA  0.071  0.051  
                 $\rho=0.5$  0.007     NA  0.034  0.007  
2dpoly  $n=500$  $\rho=0.2$  0.204     NA  0.187  0.160  
                 $\rho=0.5$  0.062     NA  0.093  0.040  
        $n=1000$ $\rho=0.2$  0.157     NA  0.134  0.109  
                 $\rho=0.5$  0.047     NA  0.064  0.024  
        $n=2000$ $\rho=0.2$  0.093     NA  0.092  0.069  
                 $\rho=0.5$  0.039     NA  0.052  0.024  
sigmoid $n=500$  $\rho=0.2$  0.034     NA  0.118  0.100  
                 $\rho=0.5$  0.029     NA  0.044  0.014  
        $n=1000$ $\rho=0.2$  0.015     NA  0.070  0.050  
                 $\rho=0.5$  0.015     NA  0.028  0.008  
        $n=2000$ $\rho=0.2$  0.001     NA  0.048  0.029  
                 $\rho=0.5$  0.003     NA  0.024  0.001  
sin     $n=500$  $\rho=0.2$  0.071     NA  0.135  0.120  
                 $\rho=0.5$  0.014     NA  0.037  0.003  
        $n=1000$ $\rho=0.2$  0.041     NA  0.082  0.065  
                 $\rho=0.5$  0.002     NA  0.025  0.007  
        $n=2000$ $\rho=0.2$  0.015     NA  0.056  0.040  
                 $\rho=0.5$  0.007     NA  0.024  0.008

In [29]:
print(res.to_latex(bold_rows=True, multirow=True,
                   multicolumn=True, escape=False,
                   column_format='lll||lll|lll|lll|lll|',
                   multicolumn_format='c|'))

\begin{tabular}{lll||lll|lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
\multirow[t]{6}{*}{\textbf{abs}} & \multirow[t]{2}{*}{\textbf{$n=500$}} & \textbf{$\rho=0.2$} & 85 & 0.187 & 0.146 & 92 & 0.949 & 0.054 & NA & 0.134 & 0.098 & NA & 0.159 & 0.137 \\
\textbf{} & \textbf{} & \textbf{$\rho=0.5$} & 99 & 0.088 & 0.013 & 98 & 1.149 & 0.003 & NA & 0.078 & 0.012 & NA & 0.053 & 0.003 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n=1000$}} & \textbf{$\rho=0.2$} & 91 & 0.125 & 0.084 & 97 & 1.591 & 0.241 & NA & 0.096 & 0.064 & NA & 0.101 & 0.085 \\
\textbf{} & \textbf{} & \textbf{$\rho=0.5$} & 100 & 0.058 & 0.003 & 100 & 0.826 & 0.089 & NA & 0.055 & 0.002 & NA & 0.036 & 0.009 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n=2000$}} & \textbf{$\rho=0.2$} & 95 & 0.084 & 0.035 & 96 & 0.